In [2]:
import os
import json
from utils.vocabulary import *

root_dir = "/scratch/yifwang/new_fairness_x_explainability/encoder_results"
models = ["bert", "roberta", "distilbert"]
bias_types = ["race", "gender", "religion"]
methods = ["Attention", "Saliency", "DeepLift", "InputXGradient", "IntegratedGradients", "Occlusion", "KernelShap"]
debiasing_methods = ["no_debiasing", "group_balance", "group_class_balance", "cda", "dropout", "attention_entropy", "causal_debias"]
training_types = ["all axes", "one axis"]
reliance_keys = ["raw", "max", "len", "norm"]
num_examples = {"race": 2000, "gender": 2000, "religion": 1000}
fairness_metrics = ["accuracy", "f1", "fpr", "fnr", "individual_fairness"]
correlation_types = ["score", "rank"]
num_val_examples_list = {"race": [50, 100, 200, 500], "gender": [50, 100, 200, 500], "religion": [50, 100, 200]}
seeds = [1, 2, 3, 4, 5, 42]

In [3]:
fairness_selection_dict = {"model": [], "bias_type": [], "fairness_metric": [], "correlation_type": [], "correlation": [], "num_val_examples": [], "seed": []}
for model in models:
    for bias_type in bias_types:
        for num_val_examples in num_val_examples_list[bias_type]:
            for seed in seeds:
                file_path = os.path.join(root_dir, "model_selection_correlation", f"{model}_{bias_type}_val_{num_val_examples}_seed_{seed}_test_{num_examples[bias_type]}", "fairness_correlation_results.json")
                with open(file_path, "r") as f:
                    results = json.load(f)
                for fairness_metric in fairness_metrics:
                    for correlation_type in correlation_types:
                        fairness_selection_dict["model"].append(model)
                        fairness_selection_dict["bias_type"].append(bias_type)
                        fairness_selection_dict["fairness_metric"].append(fairness_metric)
                        fairness_selection_dict["correlation_type"].append(correlation_type)
                        correlation_metric = fairness_metric if correlation_type == "score" else f"{fairness_metric}_rank"
                        fairness_selection_dict["correlation"].append(results[correlation_metric][0])
                        fairness_selection_dict["num_val_examples"].append(num_val_examples)
                        fairness_selection_dict["seed"].append(seed)


In [4]:
# convert to a pandas DataFrame
import pandas as pd
fairness_selection_df = pd.DataFrame(fairness_selection_dict)
#fairness_selection_df

In [7]:
for fairness_metric in fairness_metrics:
    for correlation_type in correlation_types:
        print(f"\nCorrelation by Fairness Metric: {fairness_metric}, Correlation Type: {correlation_type}")
        average_corr_df = fairness_selection_df[(fairness_selection_df["fairness_metric"] == fairness_metric) & (fairness_selection_df["correlation_type"] == correlation_type)].groupby(["bias_type", "num_val_examples"])["correlation"].mean().reset_index()
        print(average_corr_df)



Correlation by Fairness Metric: accuracy, Correlation Type: score
   bias_type  num_val_examples  correlation
0     gender                50     0.068648
1     gender               100    -0.043512
2     gender               200    -0.112389
3     gender               500    -0.128149
4       race                50     0.049603
5       race               100     0.067081
6       race               200    -0.032839
7       race               500     0.033678
8   religion                50    -0.010004
9   religion               100     0.059696
10  religion               200     0.013115

Correlation by Fairness Metric: accuracy, Correlation Type: rank
   bias_type  num_val_examples  correlation
0     gender                50     0.053724
1     gender               100     0.022222
2     gender               200     0.036386
3     gender               500     0.063736
4       race                50    -0.011233
5       race               100     0.138462
6       race               200 